In [1]:
import Estimate_Covariance_Matrix as ECM
from portfolio_choice_functions import ew_implement
from prepare_portfolio_data import define_important_dates, create_date_ranges
from Main import settings, features, pf_set
import pandas as pd
from datetime import datetime
from pandas.tseries.offsets import MonthEnd
from Prepare_Data import process_risk_free_rate, load_and_filter_market_returns_test, wealth_func
import os
import pickle
import time
import prepare_portfolio_data
import data_run_files
import Estimate_Covariance_Matrix
import Prepare_Data
import portfolio_choice_functions
from portfolio_choice_functions import w_fun, m_func
import portfolio_choice_functions
import numpy as np
from dateutil.relativedelta import relativedelta
import importlib
import General_Functions
sqrtm_cpp = importlib.import_module("sqrtm_cpp")
import time
from datetime import timedelta
from return_prediction_functions import rff
import General_Functions


import matplotlib.pyplot as plt
import seaborn as sns


Features list length: 154


In [4]:
# Path to save the portfolios.
output_path = "./data_test/"


# indhent barra_cov
barra_cov = ECM.main()
# indhent wealth
wealth = Prepare_Data.main()
wealth = wealth.dropna(subset=['wealth'])

chars, lambda_list, first_cov_date, hp_years, start_oos, date_ranges, risk_free = prepare_portfolio_data.main(barra_cov)
dates_m1 = date_ranges["dates_m1"]
dates_m2 = date_ranges["dates_m2"]
dates_oos = date_ranges["dates_oos"]
dates_hp = date_ranges["dates_hp"]


All missing excludes 0.60% of the observations
Filen er indlæst og filtreret succesfuldt.
No size screen
Turnover wo addition/deletion rule: 0.19%
Turnover w  addition/deletion rule: 33.72%


Processerer datoer: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 72/72 [00:00<00:00, 164.69it/s]


Fil indlæst med succes. Antal rækker før filtrering: 1373911
Antal ikke-NaN værdier i 'rvol_252d' i rvol_252 før merge: 2199588
Antal ikke-NaN værdier i 'rvol_252d' i df efter merge: 1353529
      id        eom  rvol_252d
0  10006 1952-08-31   0.012671
1  10006 1952-09-30   0.012712
2  10006 1952-10-31   0.011661
3  10006 1952-11-30   0.011523
4  10006 1952-12-31   0.011812
5  10006 1953-01-31   0.011484
6  10006 1953-02-28   0.011835
7  10006 1953-03-31   0.012331
8  10006 1953-04-30   0.012696
9  10006 1953-05-31   0.013877
Filen 'usa_rvol.parquet' er gemt succesfuldt i ./Data mappen!
Filen er indlæst og filtreret succesfuldt.
All missing excludes 0.60% of the observations
All missing excludes 0.60% of the observations
Filen er indlæst og filtreret succesfuldt.
No size screen
Turnover wo addition/deletion rule: 0.19%
Turnover w  addition/deletion rule: 33.72%
df_merged head:
      eom_ret     id        eom  valid_ret excntry         me   sic size_grp  \
0  1952-09-30  10006 1952-08-3

# Hjælpefunktioner

## pfml_input_fun

In [ ]:
#Portfolio-ML Inputs
def pfml_input_fun(data_tc, cov_list, lambda_list, gamma_rel, wealth, mu, dates, lb, scale,
                   risk_free, features, rff_feat, seed, p_max, g, add_orig, iter, balanced):
    # --- Lookback-datoer ---
    min_date = min(dates)
    max_date = max(dates)
    start_date = (min_date + pd.Timedelta(days=1)) - MonthEnd(lb + 1)
    dates_lb = pd.date_range(start=start_date, end=max_date, freq=MonthEnd())
    print(dates_lb)
    # --- Oprettelse af Random Fourier Features ---
    if rff_feat:
        np.random.seed(seed)
        X_features = data_tc[features].values
        rff_x = rff(X_features, p=p_max, g=g)
        rff_w = rff_x['W']
        X_cos = rff_x['X_cos']
        X_sin = rff_x['X_sin']
        rff_features = np.hstack([X_cos, X_sin])
        num = p_max // 2
        rff_colnames = [f"rff{i}_cos" for i in range(1, num + 1)] + [f"rff{i}_sin" for i in range(1, num + 1)]
        rff_df = pd.DataFrame(rff_features, columns=rff_colnames, index=data_tc.index)
        data = pd.concat([data_tc[['id', 'eom', 'valid', 'ret_ld1', 'tr_ld0', 'mu_ld0']].reset_index(drop=True),
                          rff_df.reset_index(drop=True)], axis=1)
        feat_new = list(rff_df.columns)
        if add_orig:
            data = pd.concat([data, data_tc[features].reset_index(drop=True)], axis=1)
            feat_new = feat_new + features
    else:
        cols = ['id', 'eom', 'valid', 'ret_ld1', 'tr_ld0', 'mu_ld0'] + features
        data = data_tc[cols].copy()
        feat_new = features.copy()

    # Konverter 'eom' til string-format så vi får samme format i hele funktionen
    data['eom'] = pd.to_datetime(data['eom']).dt.strftime('%Y-%m-%d')

    feat_cons = feat_new + ['constant']

    # --- Tilføjelse af scales ---
    if scale:
        scales_list = []
        for d in dates_lb:
            d_str = d.strftime('%Y-%m-%d')
            sigma = General_Functions.create_cov(cov_list[d_str])
            if hasattr(sigma, 'values'):
                sigma_vals = sigma.values
                ids_sigma = sigma.index.astype(float)
            else:
                sigma_vals = sigma
                ids_sigma = np.arange(sigma.shape[0])
            diag_vol = np.sqrt(np.diag(sigma_vals))
            df_scales = pd.DataFrame({
                'id': ids_sigma,
                'eom': d_str,  # Brug d_str for konsistens
                'vol_scale': diag_vol
            })
            scales_list.append(df_scales)
        scales_df = pd.concat(scales_list, ignore_index=True)
        data = pd.merge(data, scales_df, on=['id', 'eom'], how='left')
        data['vol_scale'] = data.groupby('eom')['vol_scale'].transform(lambda x: x.fillna(x.median()))

    # --- Justering ved balanced panel ---
    if balanced:
        for col in feat_new:
            data[col] = data.groupby('eom')[col].transform(lambda x: x - x.mean())
        data['constant'] = 1
        for col in feat_cons:
            data[col] = data.groupby('eom')[col].transform(
                lambda x: x * np.sqrt(1 / np.sum(x ** 2)) if np.sum(x ** 2) != 0 else 0)

    # --- Beregning af signaler og realiseringer for hvert dato ---
    reals_dict = {}
    signal_t_dict = {}

    for d in dates:
        if d.year % 10 == 0 and d.month == 1:
            print(f"--> PF-ML inputs: {d}")

        d_str = d.strftime('%Y-%m-%d')
        print("d", d)
        print("d_str", d_str)
        data_ret = data[(data['valid'] == True) & (data['eom'] == d_str)][['id', 'ret_ld1']]
        ids = data_ret['id'].unique()  # ids som ints
        n = len(ids)
        r_vec = data_ret['ret_ld1'].values

        # Brug de korrekte key-typer til både sigma og lambda
        sigma = General_Functions.create_cov(cov_list[d_str], ids=ids)
        lambda_series = pd.Series(lambda_list[d_str])
        lambda_mat = General_Functions.create_lambda(lambda_series, ids=ids)

        w = wealth.loc[wealth['eom'] == d_str, 'wealth'].iloc[0]
        rf = risk_free.loc[risk_free['eom'] == d_str, 'rf'].iloc[0]
        m = m_func(w=w, mu=mu, rf=rf, sigma_gam=sigma * gamma_rel, gam=gamma_rel,
                   lambda_mat=lambda_mat, iter=iter)

        # Brug MonthEnd til at beregne lower_bound – konverter til string-format
        lower_bound = (d - MonthEnd(lb)).strftime('%Y-%m-%d')
        data_sub = data[(data['id'].isin(ids)) & (data['eom'] >= lower_bound) & (data['eom'] <= d_str)].copy()

        if not balanced:
            for col in feat_new:
                data_sub[col] = data_sub.groupby('eom')[col].transform(lambda x: x - x.mean())
            data_sub['constant'] = 1
            for col in feat_cons:
                data_sub[col] = data_sub.groupby('eom')[col].transform(
                    lambda x: x * np.sqrt(1 / np.sum(x ** 2)) if np.sum(x ** 2) != 0 else 0)

        data_sub = data_sub.sort_values(by=['eom', 'id'], ascending=[False, True])
        # Gruppér efter 'eom' – vi konverterer gruppenøglerne til string-format for konsistens
        groups = {k: group for k, group in data_sub.groupby('eom')}

        # Beregn signaler med diagnose: udskriv kun hvis NaN, Inf, tom DataFrame eller tomt index
        signals = {}
        for eom_val, group in groups.items():
            if group.empty or group.index.empty:
                print(f"Advarsel: Gruppe for {eom_val} er tom!")
            s = group[feat_cons].values
            if scale:
                s = np.diag(1 / group['vol_scale'].values) @ s
            # Tjek for NaN eller Inf
            if np.isnan(s).any():
                print(f"Advarsel: Signal for {eom_val} indeholder NaN. Form: {s.shape}")
            if np.isinf(s).any():
                print(f"Advarsel: Signal for {eom_val} indeholder Inf. Form: {s.shape}")
            signals[eom_val] = s
        print("signaler", signals)
        d_key = d_str  # d_str er allerede i '%Y-%m-%d'-format
        signal_current = signals.get(d_key, None)
        if signal_current is None:
            print(f"Advarsel: Ingen signal fundet for {d_key}.")
            continue
        elif np.isnan(signal_current).any() or np.isinf(signal_current).any():
            print(f"Advarsel: Signal for {d_key} indeholder NaN eller Inf, springer denne dato over.")
            continue

        # Beregn gtm for hver gruppe
        gtm = {}
        for eom_val, group in groups.items():
            gt = (1 + group['tr_ld0']) / (1 + group['mu_ld0'])
            gt = gt.fillna(1).values
            gtm[eom_val] = m @ np.diag(gt)

        # Aggregér gtm over lookback-perioden
        n_stocks = n
        gtm_agg = {}
        gtm_agg_l1 = {}
        gtm_agg[d_key] = np.eye(n_stocks)
        gtm_agg_l1[d_key] = np.eye(n_stocks)
        for i in range(1, lb + 1):
            d_i = (d - MonthEnd(i)).strftime('%Y-%m-%d')
            # DEBUG: Udskriv unikke 'eom'-datoer for denne lookback-dato
            lookback_date = pd.to_datetime(d_i)
            unique_dates = data[data['eom'] == lookback_date.strftime('%Y-%m-%d')]['eom'].unique()
            print(f"Lookup for lookback-dato: {d_i} - Unikke eom-datoer fundet: {unique_dates}")

            if d_i in gtm:
                gtm_agg[d_i] = gtm_agg[list(gtm_agg.keys())[-1]] @ gtm[d_i]
            else:
                gtm_agg[d_i] = gtm_agg[list(gtm_agg.keys())[-1]]
            d_i_l1 = (d - MonthEnd(i + 1)).strftime('%Y-%m-%d')
            if d_i_l1 in gtm:
                gtm_agg_l1[d_i] = gtm_agg_l1[list(gtm_agg_l1.keys())[-1]] @ gtm[d_i_l1]
            else:
                gtm_agg_l1[d_i] = gtm_agg_l1[list(gtm_agg_l1.keys())[-1]]

        # Summering over lookback: opbyg omega og konstanter
        omega_sum = None
        const_sum = None
        omega_l1_sum = None
        const_l1_sum = None
        for i in range(0, lb + 1):
            d_i = (d - MonthEnd(i)).strftime('%Y-%m-%d')
            s_i = signals.get(d_i, None)
            if s_i is None:
                print(f"Advarsel: Ingen signal fundet for lookback-dato d_i {d_i}")
                term = np.zeros((n_stocks, len(feat_cons)))
            else:
                term = gtm_agg.get(d_i, np.eye(n_stocks)) @ s_i
            if omega_sum is None:
                omega_sum = term
                const_sum = gtm_agg.get(d_i, np.eye(n_stocks))
            else:
                omega_sum = omega_sum + term
                const_sum = const_sum + gtm_agg.get(d_i, np.eye(n_stocks))

            d_i_l1 = (d - MonthEnd(i + 1)).strftime('%Y-%m-%d')
            s_i_l1 = signals.get(d_i_l1, None)
            if s_i_l1 is None:
                print(f"Advarsel: Ingen signal fundet for lookback-dato d_i_l1 {d_i_l1}")
                term_l1 = np.zeros((n_stocks, len(feat_cons)))
            else:
                term_l1 = gtm_agg_l1.get(d_i_l1, np.eye(n_stocks)) @ s_i_l1
            if omega_l1_sum is None:
                omega_l1_sum = term_l1
                const_l1_sum = gtm_agg_l1.get(d_i_l1, np.eye(n_stocks))
            else:
                omega_l1_sum = omega_l1_sum + term_l1
                const_l1_sum = const_l1_sum + gtm_agg_l1.get(d_i_l1, np.eye(n_stocks))

        # Løs de lineære systemer
        omega_final = np.linalg.solve(const_sum, omega_sum)
        omega_l1_final = np.linalg.solve(const_l1_sum, omega_l1_sum)

        # Beregn gt for den aktuelle gruppe (d)
        if d_key in signals:
            group_d = groups.get(d_key, None)
            if group_d is None:
                gt_vec = np.ones(n_stocks)
            else:
                gt_vec = ((1 + group_d['tr_ld0']) / (1 + group_d['mu_ld0'])).values
            gt_mat = np.diag(gt_vec)
        else:
            gt_mat = np.eye(n_stocks)

        omega_chg = omega_final - gt_mat @ omega_l1_final

        # --- Realiseringer ---
        r_tilde = omega_final.T @ r_vec
        risk_val = gamma_rel * (omega_final.T @ sigma @ omega_final)
        tc_val = w * (omega_chg.T @ lambda_mat @ omega_chg)
        denom = risk_val + tc_val

        reals = {
            "r_tilde": r_tilde.item() if np.isscalar(r_tilde) or r_tilde.size == 1 else r_tilde,
            "denom": denom.item() if np.isscalar(denom) or denom.size == 1 else denom,
            "risk": risk_val.item() if np.isscalar(risk_val) or risk_val.size == 1 else risk_val,
            "tc": tc_val.item() if np.isscalar(tc_val) or tc_val.size == 1 else tc_val
        }

        reals_dict[d_key] = reals
        signal_t_dict[d_key] = signals.get(d_key, None)

    return {"reals": reals_dict, "signal_t": signal_t_dict, "rff_w": rff_w if rff_feat else None}

## pfml_feat_fun

In [ ]:
# Feature names
def pfml_feat_fun(p, orig_feat, features=None):
    """
    Returnerer en liste af feature-navne.

    Parametre:
      p         : Antal RFF-features (skal være deleligt med 2). Hvis p er 0, bruges ingen RFF-features.
      orig_feat : Bool – om de originale features skal inkluderes.
      features  : Liste af originale feature-navne. Skal gives, hvis orig_feat er True.

    Returnerer:
      En liste med feature-navne.
    """
    feat = ["constant"]
    if p != 0:
        half_p = p // 2
        feat.extend([f"rff{i}_cos" for i in range(1, half_p + 1)])
        feat.extend([f"rff{i}_sin" for i in range(1, half_p + 1)])
    if orig_feat and features is not None:
        feat.extend(features)
    return feat


## denom_sum_fun

In [ ]:
# Compute mean of denom matrices before hp_years
def denom_sum_fun(train):
    """
    Beregner summen af 'denom'-værdier fra en liste af træningsdata.
    
    Parametre:
      train : Liste af ordbøger, hvor hver ordbog indeholder en nøgle 'denom'.
      
    Returnerer:
      Summen af alle 'denom'-værdier.
    """
    # Hvis 'denom' er et tal eller en numpy-array, summeres de med sum().
    return sum(x['denom'] for x in train)

## pfml_search_coef

In [ ]:
# Hyper search for best coefficients ---------------------
def pfml_search_coef(pfml_input, p_vec, l_vec, hp_years, orig_feat):
    # Konverter nøglerne (datoer) til datetime-objekter
    reals = pfml_input["reals"]
    # Vi antager, at nøglerne kan konverteres med pd.to_datetime
    d_all = {pd.to_datetime(k): k for k in reals.keys()}
    
    # Bestem end_bef = min(hp_years) - 1 (fx hvis det mindste hp_year er 2016, så er end_bef 2015)
    end_bef = 2021  # - 1
    # Vælg træningsdata før: datoer < (min(hp_years)-2)-12-31
    cutoff_date = pd.to_datetime(f"{end_bef - 1}-12-31")
    train_bef = {k: reals[d_all[k]] for k in d_all if k < cutoff_date}
    
    # Beregn sum af r_tilde over træningssættet.
    # Her antages det, at hver x['r_tilde'] er en pandas Series med index svarende til feature-navne.
    r_tilde_list = [item['r_tilde'] for item in train_bef.values()]
    # Summering: her benyttes sum() til pandas Series (forudsætter, at listen ikke er tom)
    r_tilde_sum = sum(r_tilde_list) if r_tilde_list else pd.Series(0)
    
    # Tilsvarende: sum af 'denom'-matricer (antages at være pandas DataFrames med feature-navne som index/columns)
    denom_list = [item['denom'] for item in train_bef.values()]
    denom_raw_sum = sum(denom_list) if denom_list else pd.DataFrame(0)
    
    # n er antallet af observationer i train_bef
    n = len(train_bef) # skal evt ændres
    
    # Sorter hp_years, og opret en ordbog til at holde koefficienterne
    hp_years = sorted(hp_years)
    coef_list = {}
    
    # For hvert hp_year
    for hp in hp_years:
        # Udvælg nye træningsobservationer: datoer i intervallet
        # [ (hp-2)-12-31, (hp-1)-11-30 ]
        lower_bound = pd.to_datetime(f"{hp - 2}-12-31")
        upper_bound = pd.to_datetime(f"{hp - 1}-11-30")
        
        train_new = {k: reals[d_all[k]] for k in d_all if lower_bound <= k <= upper_bound}
        
        # Opdater antallet af observationer
        n += len(train_new)
        # Opdater r_tilde_sum med de nye r_tilde-værdier
        r_tilde_new = sum([item['r_tilde'] for item in train_new.values()]) if len(train_new) > 0 else 0
        r_tilde_sum = r_tilde_sum + r_tilde_new
    
        
    # Opdater denom-summen
    denom_raw_new = sum([item['denom'] for item in train_new.values()]) if len(train_new) > 0 else 0
    denom_raw_sum = denom_raw_sum + denom_raw_new
    
    # For hvert p i p_vec, beregn de tilhørende koefficienter
    coef_by_hp = {}
    # Hent feature-navnene givet p og om de originale features skal medtages
    feat_p = pfml_feat_fun(p, orig_feat)
    all_feat = feat_p
    # r_tilde_sub: udtræk de features fra r_tilde_sum og divider med n
    r_tilde_sum = pd.Series(r_tilde_sum, index=all_feat)
    
    r_tilde_sub = r_tilde_sum.loc[feat_p] / n
    # denom_sub: udtræk de relevante rækker og kolonner fra denom_raw_sum og divider med n
    denom_raw_sum.index = all_feat
    denom_raw_sum.columns = all_feat
    
    denom_sub = denom_raw_sum.loc[feat_p, feat_p] / n
        
    results = {}
    for l in l_vec:
        M = denom_sub + l * np.eye(len(feat_p))
        # Løs det lineære system: M * coef = r_tilde_sub
        coef = np.linalg.solve(M.values, r_tilde_sub.values)
        # Gem resultatet som en pandas Series med index = feat_p
        results[l] = pd.Series(coef, index=feat_p)
    # Gem resultaterne for den aktuelle p-værdi
    coef_by_hp[p] = results
    # Gem koefficienterne for det aktuelle hp_year, brug hp som streng
    coef_list[str(hp)] = coef_by_hp
        
    return coef_list

## pfml_hp_reals_fun

In [ ]:
# Next, compute realized utility for each p in each hp_year -------------------
def pfml_hp_reals_fun(pfml_input, hp_coef, p_vec, l_vec, hp_years, orig_feat):

    # Konverter nøglerne for pfml_input["reals"] til pd.Timestamp og opret en mapping
    reals_all_raw = pfml_input["reals"]
    reals_all_dict = {pd.to_datetime(k): v for k, v in reals_all_raw.items()}

    # Liste til at samle alle rækker
    rows = []

    # For hvert hp_year (bruges som 'end')
    for end in sorted(hp_years):
        # Vælg observationer med datoer: >= (end-1)-12-31 og <= end-11-30
        lower_bound = pd.to_datetime(f"{end - 1}-12-31")
        upper_bound = pd.to_datetime(f"{end}-11-30")
        # Filtrer reals for dette hp_year
        reals_all = {dt: obj for dt, obj in reals_all_dict.items() if lower_bound <= dt <= upper_bound}
        
        # Hent koefficienterne for dette hp_year
        coef_list_yr = hp_coef[str(end)]
        
        # For hvert p i p_vec
        for p in p_vec:
            # Hent de relevante feature-navne for p
            feat_p = pfml_feat_fun(p, orig_feat)  # Forudsætter at funktionen er defineret
            # Hent koefficientlisten for denne p-værdi (nøglen som string)
            coef_list_p = coef_list_yr[str(p)]
            
            # For hver observation i reals_all: udtræk r_tilde og denom for feat_p
            reals_subset = {}
            for dt, obj in reals_all.items():
                # Restriktion af r_tilde og denom til de features, der er i feat_p
                r_tilde_sub = obj["r_tilde"][feat_p]
                denom_sub = obj["denom"].loc[feat_p, feat_p]
                reals_subset[dt] = {"r_tilde": r_tilde_sub, "denom": denom_sub}
            
            # For hver l-værdi (brug indeks i l_vec)
            for i, l_value in enumerate(l_vec):
                # Hent koefficienten for denne l-værdi
                coef = coef_list_p[i]  # Forventet at være en pandas Series med index = feat_p
                # For hver dato (nøgle) i reals_subset, beregn realiseret nytte
                for dt, sub_obj in reals_subset.items():
                    # Beregn: r = t(r_tilde) %*% coef - 0.5 * t(coef) %*% denom %*% coef
                    r_val = sub_obj["r_tilde"].dot(coef) - 0.5 * (coef.dot(sub_obj["denom"].dot(coef)))
                    # Beregn eom_ret: her beregnes som månedsslut for næste måned
                    eom_ret = dt + MonthEnd(1)
                    # Gem række med de relevante oplysninger
                    rows.append({
                        "eom": dt,
                        "eom_ret": eom_ret,
                        "obj": r_val,
                        "l": l_value,
                        "p": p,
                        "hp_end": end
                    })

    # Sammensæt resultaterne til et DataFrame
    validation = pd.DataFrame(rows)
    
    # Sorter efter p, l og eom_ret
    validation = validation.sort_values(by=["p", "l", "eom_ret"]).reset_index(drop=True)
    
    # Beregn kumulativ middelværdi af 'obj' for hver gruppe (p, l) i kronologisk rækkefølge af eom_ret.
    validation["cum_obj"] = validation.groupby(["p", "l"])["obj"].transform(lambda x: x.expanding().mean())
    
    # For hver eom_ret, beregn rangeringen af -cum_obj (dvs. højere cum_obj giver lavere ranknummer)
    validation["rank"] = validation.groupby("eom_ret")["cum_obj"].rank(ascending=False)
    
    return validation


## pfml_aims_fun 

In [ ]:
# Create optimal aim portfolio (for a given g) -----------------
def pfml_aims_fun(pfml_input, validation, data_tc, hp_coef, hp_years, dates_oos, l_vec, orig_feat):
    """
    Opretter den optimale AIM-portefølje (for et g) baseret på de fundne hyperparametre.
    
    Parametre:
      pfml_input : Dictionary med nøglen "signal_t". Her skal pfml_input["signal_t"] være en dictionary,
                   hvor nøglerne er dato-strenge (fx "2020-01-31") og værdier er DataFrames med signaler.
      validation : Pandas DataFrame med bl.a. kolonnerne 'eom_ret', 'rank', 'l' og 'p'.
                   Her antages 'eom_ret' at være datetime, så vi kan udlede året.
      data_tc    : Pandas DataFrame med portefølje-data, indeholdende mindst 'id', 'eom' og 'valid'.
      hp_coef    : Dictionary med koefficienter for hvert out-of-sample år. Strukturen er:
                   hp_coef[str(oos_year)][str(p)][l_index] = koefficient (pandas Series).
      hp_years   : Liste med hp-år (fx [2016, 2017, ...]).
      dates_oos  : Liste af out-of-sample datoer (pd.Timestamp objekter).
      l_vec      : Liste med lambda-værdier.
      orig_feat  : Boolean, angivende om de originale features skal medtages.
      
    Returnerer:
      aim_pfs_list : Dictionary med nøgler svarende til hver dato i dates_oos, hvor hver værdi er en dictionary med nøglerne:
                     "aim_pf" : DataFrame med kolonnerne 'id', 'eom' og 'w_aim'.
                     "coef"   : Den valgte koefficient (pandas Series) for den aktuelle dato.
    """
    # Udvælg optimum hyperparametre: de rækker hvor eom_ret er i december og rank==1,
    # og udled hp_end som året fra eom_ret.
    validation = validation.copy()
    validation['hp_end'] = validation['eom_ret'].dt.year
    opt_hps = validation[(validation['eom_ret'].dt.month == 12) & (validation['rank'] == 1)][['hp_end', 'l', 'p']]
    
    print("Optimum hyperparametre (opt_hps):")
    print(opt_hps)
    
    aim_pfs_list = {}
    
    for d in dates_oos:
        # Udregn d_ret svarende til: d + 1 + måneder(1) - 1 
        # (dvs. at få månedsslut for den næste måned)
        d_ret = d + pd.Timedelta(days=1) + DateOffset(months=1) - pd.Timedelta(days=1)
        print("d_ret:")
        print(d_ret)
        
        oos_year = d_ret.year
        print("oos_year:")
        print(oos_year)
        
        # Vælg hp_year: hvis (oos_year - 1) findes blandt opt_hps.hp_end, så brug det,
        # ellers bruges den mindste værdi af opt_hps.hp_end.
        if (oos_year - 1) in opt_hps['hp_end'].values:
            hp_year = oos_year - 1
        else:
            hp_year = opt_hps['hp_end'].min()
        print("hp_year:")
        print(hp_year)
        
        # Vælg de hyperparametre, der matcher hp_year
        hps_d = opt_hps[opt_hps['hp_end'] == hp_year]
        print("hps_d:")
        print(hps_d)
        
        if hps_d.empty:
            print(f"Ingen hyperparameter fundet for hp_year: {hp_year} til dato: {d_ret}")
            continue
        else:
            print(f"Hyperparameter for hp_year {hp_year}:")
            print(hps_d)
        
        # Antag at der kun er én række – ellers kan man tage den første
        p_val = hps_d['p'].iloc[0]
        print(p_val)
        l_val = hps_d['l'].iloc[0]
        print(l_val)
        # Hent de relevante features via pfml_feat_fun (skal være defineret)
        feat = pfml_feat_fun(p=p_val, orig_feat=orig_feat)
        
        # Hent signalerne for dato d. Vi antager, at nøglerne i pfml_input["signal_t"] er i ISO-format.
        key_d = d.strftime('%Y-%m-%d')
        s = pfml_input["signal_t"][key_d][feat]
        
        # Find indeks for l_val i l_vec (brug evt. en tilnærmelsesmetode for floating point)
        try:
            l_no = l_vec.index(l_val)
        except ValueError:
            l_no = min(range(len(l_vec)), key=lambda i: abs(l_vec[i] - l_val))
        
        # Hent den korrekte koefficient
        key_oos_year = str(oos_year)
        key_p = str(p_val)
        coef = hp_coef[key_oos_year][key_p][l_no]
        
        # Filtrer data_tc for rækker, hvor valid==True og eom == d
        aim_pf_data = data_tc[(data_tc['valid'] == True) & (data_tc['eom'] == d)].copy()
        # Beregn w_aim = s %*% coef. Her antages det, at s er en DataFrame med samme rækkefølge som
        # de relevante rækker i data_tc, og at coef er en pandas Series med index svarende til kolonnerne i s.
        aim_pf_data['w_aim'] = s.dot(coef)
        aim_pf = aim_pf_data[['id', 'eom', 'w_aim']]
        
        aim_pfs_list[d] = {"aim_pf": aim_pf, "coef": coef}
    
    return aim_pfs_list

## pfml_w  mangler

# Implementation FUNCTION SIDSTE

In [2]:
def pfml_implement(data_tc, cov_list, lambda_list, risk_free, features, 
                   wealth, mu, gamma_rel, dates_full, dates_oos, lb, 
                   hp_years, rff_feat, scale, 
                   g_vec=None, p_vec=None, l_vec=None, orig_feat=None, iter=None, hps=None, balanced=None, seed=None):
    #data_tc = data_tc[(data_tc['valid'] == True) & (data_tc['eom'].isin(dates_oos))].copy() # forsøg med dato
    # Hyperparameter search ----------
    if hps is None:
        start_time = time.time()
        hps = {}
        # Loop over hvert g i g_vec
        for g in g_vec:
            start_g = time.time()
            # Step 1: Prepare data
            # Antag at pfml_input_fun returnerer en dictionary med nøglerne 'rff_w', 'reals' og 'signal_t'
            pfml_input = pfml_input_fun(
                data_tc, wealth=wealth, mu=mu, dates=dates_full,
                lb=lb, scale=scale, risk_free=risk_free,
                cov_list=cov_list, lambda_list=lambda_list, gamma_rel=gamma_rel, iter=iter,
                features=features, balanced=balanced,
                rff_feat=rff_feat, p_max=max(p_vec), g=g, add_orig=orig_feat, seed=seed
            )
            print("pfml_input_fun took {:.2f} sec".format(time.time()-start_g))
            # RFF weights
            rff_w = pfml_input['rff_w']
            # Inputs – antag at pfml_feat_fun returnerer en liste/array med de ønskede indeks
            feat_all = pfml_feat_fun(p=max(p_vec), orig_feat=orig_feat)
            # Juster pfml_input, så de relevante matricer/vektorer kun indeholder de kolonner, der svarer til feat_all
            pfml_input = adjust_pfml_input(pfml_input, feat_all)
            # (Bemærk: Linjen nedenfor fjernes, da adjust_pfml_input allerede håndterer signal_t)
            # pfml_input['signal_t'] = {k: x[:, feat_all] for k, x in pfml_input['signal_t'].items()}
            
            # Coefficients – antag at pfml_search_coef er en Python‑funktion
            pfml_hp_coef = pfml_search_coef(pfml_input, p_vec=p_vec, l_vec=l_vec, hp_years=hp_years, orig_feat=orig_feat)
            # Realized utility – antag at pfml_hp_reals_fun returnerer en DataFrame
            validation = pfml_hp_reals_fun(pfml_input, hp_coef=pfml_hp_coef, p_vec=p_vec, l_vec=l_vec, hp_years=hp_years, orig_feat=orig_feat)
            validation['g'] = g  # Tilføj kolonnen g med den nuværende værdi
            # Find optimal aim portfolio – antag at pfml_aims_fun er implementeret
            print("--------------------------------")
            aims = pfml_aims_fun(pfml_input, validation=validation, data_tc=data_tc, hp_coef=pfml_hp_coef, hp_years=hp_years, dates_oos=dates_oos, l_vec=l_vec, orig_feat=orig_feat)
            elapsed_g = (time.time() - start_g) / 60
            print(f"g: {g:.2f} took {elapsed_g:.2f} min")
            # Gem resultaterne for dette g
            hps[str(g)] = {"aim_pfs_list": aims, "validation": validation, "rff_w": rff_w}
        print("Total hyperparameter search took {:.2f} sec".format(time.time()-start_time))

    print("hps efter kørsel", hps)
    # Implement final portfolio -----------------
    # Kombinér alle validation-DataFrames fra hps til én DataFrame (uden 'rank'-kolonnen)
    validation_list = [hp_dict["validation"] for hp_dict in hps.values()]
    print("validation_list", validation_list)
    best_hps = pd.concat(validation_list, ignore_index=True)
    print("best_hps", best_hps)
    if 'rank' in best_hps.columns:
        best_hps = best_hps.drop(columns='rank')
    # Beregn rank pr. eom_ret-gruppe: rank baseret på -cum_obj (højere cum_obj = lavere rank)
    best_hps['rank'] = best_hps.groupby('eom_ret')['cum_obj'].rank(ascending=False, method='min')
    # Vælg de rækker, hvor rank == 1 og eom_ret er i december
    best_hps['eom_ret'] = pd.to_datetime(best_hps['eom_ret'])
    best_hps = best_hps[(best_hps['rank'] == 2.5) & (best_hps['eom_ret'].dt.month == 12)]
    
    # Tjek at hyperparameter-området er passende: Lav et plot af parametrene p, l og g mod eom_ret
    melt_df = best_hps.melt(id_vars=['eom_ret'], value_vars=['p', 'l', 'g'], var_name='param', value_name='value')
    # plt.figure(figsize=(10, 6))
    # sns.scatterplot(data=melt_df, x='eom_ret', y='value', hue='param', alpha=0.5)
    # sns.lineplot(data=melt_df, x='eom_ret', y='value', hue='param')
    # plt.title("HP Range Check")
    # plt.show()
    
    # Aim portfolios: For hver dato i dates_oos
    best_hps_list = {}
    for d in dates_oos:
        # d_ret beregnes som den sidste dag i den næste måned:
        d_ret = d + pd.offsets.MonthEnd(1)
        oos_year = d_ret.year
        # Vælg rækker i best_hps for år = oos_year - 1
        print(best_hps)
        bh = best_hps[best_hps['eom_ret'].dt.year == (oos_year)] #== (oos_year - 1)]
        if bh.empty:
            continue
        best_g = bh['g'].iloc[0]
        best_p = bh['p'].iloc[0]
        d_str = d.strftime('%Y-%m-%d')
        # Hent best_aim og best_coef fra hps; antag at aim_pfs_list er en dictionary med nøgler svarende til datoer (som strenge)
        print("Aim_pfs_list keys for best_g:", list(hps[str(best_g)]["aim_pfs_list"].keys()))
        print("Lookup key d_str:", d_str)

        best_aim = hps[str(best_g)]["aim_pfs_list"][d_str]["aim_pf"]
        best_coef = hps[str(best_g)]["aim_pfs_list"][d_str]["coef"]
        best_hps_list[d_str] = {"g": best_g, "p": best_p, "aim": best_aim, "coef": best_coef}
    
    # Kombinér alle aim-portfolios fra best_hps_list til en DataFrame
    aims_list = [item["aim"] for item in best_hps_list.values()]
    aims = pd.concat(aims_list, ignore_index=True)
    
    # Final portfolio: Vælg data_tc for datoer i dates_oos og hvor valid == True
    w_data = data_tc[(data_tc['valid'] == True) & (data_tc['eom'].isin(dates_oos))][['id', 'eom', 'eom_ret', 'me', 'tr_ld1', 'valid']].copy()
    # Antag at pfml_w er en Python‑funktion, der returnerer en DataFrame med de beregnede vægte
    w = pfml_w(w_data, dates=dates_oos, cov_list=cov_list, lambda_list=lambda_list, gamma_rel=gamma_rel, iter=iter, risk_free=risk_free, wealth=wealth, mu=mu, aims=aims)
    
    pf = pf_ts_fun(w, data=data_tc, wealth=wealth, gam=gamma_rel)
    pf['type'] = "Portfolio-ML"
    
    # List of Ws used: Lav en dictionary med nøgler fra g_vec og værdier fra hps['rff_w']
    rff_w_list = {g: hps[str(g)]["rff_w"] for g in g_vec}
    
    return {"hps": hps, "best_hps": best_hps, "best_hps_list": best_hps_list, "aims": aims, "w": w, "pf": pf, "rff_w_list": rff_w_list}
